In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scvi
import scanpy as sc
import matplotlib
import csv
import os
import seaborn as sns
import gseapy
from gseapy import dotplot, ringplot

seed=10
os.chdir(".../Atlas/objects")
sc.logging.print_versions()
!pip list

Global seed set to 0


-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                 9.2.0
absl                NA
asttokens           NA
attr                22.1.0
backcall            0.2.0
bottleneck          1.3.5
certifi             2022.09.14
cffi                1.15.1
chardet             5.1.0
charset_normalizer  2.1.1
chex                0.1.5
colorama            0.4.5
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.3
decorator           4.4.2
defusedxml          0.7.1
deprecate           0.3.2
docrep              0.3.2
entrypoints         0.4
etils               0.8.0
executing           1.0.0
flax                0.6.0
fsspec              2022.8.2
google              NA
gseapy              1.0.4
h5py                3.7.0
idna                3.4
igraph              0.9.11
ipykernel           6.15.3
ipython_genutils    0.2.0
ipywidgets          8.0.2
jax                 0.3.17
jaxlib              0.3.15
jedi                0.18.0
joblib          

In [2]:
adata_meta = sc.read("Atlas6.6_3000HVG_integrated_V2.h5ad")

In [ ]:
adata = sc.read("Atlas6.4_raw.h5ad")
adata

In [4]:
adata.obs["annotation_final_level1B"] = adata_meta.obs["annotation_final_level1B"]

In [7]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)

In [ ]:
os.chdir(".../Atlas/DEGs_species/level_1_all")


celltypes = adata.obs.annotation_final_level1.unique().tolist()
print(celltypes)

# Loop through each cell type
for n, celltype in enumerate(celltypes):
    # Subset the adata object based on the cell type
    adata_subset = adata[adata.obs.annotation_final_level1 == celltype]

    
    sc.tl.rank_genes_groups(adata_subset, 'species', method='wilcoxon', key_added="wilcoxon")


    # Human
    human_df = sc.get.rank_genes_groups_df(adata_subset, group='human', 
                                    key='wilcoxon', pval_cutoff=0.05)
    
    #Mouse
    
    mouse_df = sc.get.rank_genes_groups_df(adata_subset, group='mouse', 
                                    key='wilcoxon', pval_cutoff=0.05)
    
    #rat
    
    rat_df = sc.get.rank_genes_groups_df(adata_subset, group='rat', 
                                    key='wilcoxon', pval_cutoff=0.05)

    
    human_df.to_csv(f'human_DEG_{celltype}.csv', index=False)
    mouse_df.to_csv(f'mouse_DEG_{celltype}.csv', index=False)
    rat_df.to_csv(f'rat_DEG_{celltype}.csv', index=False)

In [ ]:
os.chdir(".../Atlas/DEGs_species/level_1B_all")


celltypes = adata.obs.annotation_final_level1B.unique().tolist()
print(celltypes)


for n, celltype in enumerate(celltypes):
    # Subset the adata object based on the cell type
    adata_subset = adata[adata.obs.annotation_final_level1B == celltype]
 
    sc.tl.rank_genes_groups(adata_subset, 'species', method='wilcoxon', key_added="wilcoxon")


    # Human
    human_df = sc.get.rank_genes_groups_df(adata_subset, group='human', 
                                    key='wilcoxon', pval_cutoff=0.05)
    
    #Mouse
    
    mouse_df = sc.get.rank_genes_groups_df(adata_subset, group='mouse', 
                                    key='wilcoxon', pval_cutoff=0.05)
    
    #rat
    
    rat_df = sc.get.rank_genes_groups_df(adata_subset, group='rat', 
                                    key='wilcoxon', pval_cutoff=0.05)

    
    
    
    
    
    human_df.to_csv(f'human_DEG_{celltype}.csv', index=False)
    mouse_df.to_csv(f'mouse_DEG_{celltype}.csv', index=False)
    rat_df.to_csv(f'rat_DEG_{celltype}.csv', index=False)
    
    


In [ ]:
order = ["Podo", "PEC", "PTS1", "PTS2", "PTS3", "injPT", 
         "prolif_Tubule", "DTL_ATL", "MD", "TAL", "injTAL",
         "DCT", "DCT2", "CNT", "CD_PC", "injDCT_CNT", "ICA", "ICB", 
         "Immune", "Stromal", "EC"]

In [ ]:
os.chdir(".../Atlas/DEGs_species/level_1B_all")


celltypes = order

# empty list to store top 3 DEGs for each cell type
top_degs = []

# loop through cell types and extract top 3 DEGs
for celltype in celltypes:
    file_path = f"human_DEG_{celltype}.csv"
    df = pd.read_csv(file_path)
    top_genes = df.iloc[:3, 0].tolist()
    top_degs.extend(top_genes)
    
    file_path = f"mouse_DEG_{celltype}.csv"
    df = pd.read_csv(file_path)
    top_genes = df.iloc[:3, 0].tolist()
    top_degs.extend(top_genes)
    
    file_path = f"rat_DEG_{celltype}.csv"
    df = pd.read_csv(file_path)
    top_genes = df.iloc[:3, 0].tolist()
    top_degs.extend(top_genes)

print(top_degs)

In [ ]:
# Example lists for cell types and species
cell_types = order  
species = ['human', 'mouse', 'rat']  


new_order = []

# Loop through each combination of cell type and species and add them to the new order
for ct in cell_types:
    for sp in species:
        combined_name = f"{ct}_{sp}"
        new_order.append(combined_name)



In [ ]:
#dot plot with marker genes
sc.pl.dotplot(
adata,
top_degs,
groupby=['annotation_final_level1B', 'species'],
dendrogram=False,
color_map="Blues",
swap_axes=True,
standard_scale="var",
categories_order = new_order
)